# Perceptron e programmazione a oggetti - inizio parte 1

In [ ]:
import pandas as pd
import numpy as np
path = "C:\\Users\\ianto\\Desktop\\Corso Python per l analisi dei dati\\File\\"

In [ ]:
def preprocessing(path, soglia_null = 0.5, percentuale_training = 0.7):
    iris = pd.read_csv(filepath_or_buffer=path + 'IrisDataset.csv',
            sep = ";", 
            header = None,
            names = ["rownumber","sepal_length","sepal_width","petal_length","petal_width","class"],
            skiprows = 4
            )
    
    iris_working = iris.copy()
    iris_working = iris_working[iris_working["class"].isin(["Iris setosa","Iris versicolor"])]
    iris_working = iris_working.sample(frac=1, 
                                       random_state = 0)
    iris_working = iris_working.drop(["rownumber"], 
                                     axis=1)
    
    
    numero_righe = len(iris_working)
    soglia = numero_righe * soglia_null
    lista_colonne = list(iris_working.columns)
    lista_colonne.remove("class")
    for colonna in lista_colonne:
        numero_null = len(iris_working[iris_working[colonna].isna()])
        if numero_null > soglia:
            iris_working.drop(colonna, axis=1)     
    
    iris_working.loc[iris_working["class"] == "Iris setosa", "class"] = 1
    iris_working.loc[iris_working["class"] == "Iris versicolor", "class"] = -1
    
    numero_righe_training = int(numero_righe*percentuale_training)
    iris_training = iris_working.iloc[0:numero_righe_training].copy()
    iris_test = iris_working.iloc[numero_righe_training:].copy()
    
    for colonna in lista_colonne:
        media = iris_training[colonna].mean()
        iris_training[colonna] = iris_training[colonna].fillna(value = media)
        iris_test[colonna] = iris_test[colonna].fillna(value = media)
        
    for colonna in lista_colonne:
        media = iris_training[colonna].mean()
        deviazione_standard = iris_training[colonna].std()
        iris_training[colonna] = (iris_training[colonna] - media) / deviazione_standard
        iris_test[colonna] = (iris_test[colonna] - media) / deviazione_standard
            
    iris_training["weight"] = 1
    iris_test["weight"] = 1
    
    x1 = iris_training[["sepal_length","sepal_width","petal_length","petal_width","weight"]].values
    y1 = iris_training[["class"]].values

    x2 = iris_test[["sepal_length","sepal_width","petal_length","petal_width","weight"]].values
    y2 = iris_test[["class"]].values

    return [iris, iris_training, iris_test, x1, y1, x2, y2]

In [ ]:
def predici(x,w):
    x_dots_w = np.dot(x,w) 
    if x_dots_w >= 0:
        predizione = 1
    else:
        predizione = -1
    return predizione

In [ ]:
def perceptron(x,y,eta=0.1,epoche=10):
    w = np.zeros(x.shape[1])
    iterazioni_eseguite = 0
    lista_errori = []
    for i1 in range(epoche): 
        iterazioni_eseguite = iterazioni_eseguite + 1
        errori = 0
        for i in range(x.shape[0]):
            predizione = predici(x[i],w)
            w = w + eta * (y[i]-predizione) * x[i]
            if (y[i]-predizione) != 0:
                errori = errori + 1
        lista_errori.append(errori)
        if(errori == 0):
            break
    return [w, lista_errori]

# Eseguiamo le due funzioni preprocessing e perceptron

In [ ]:
iris, iris_training, iris_test, x1, y1, x2, y2 = preprocessing(path = "C:\\Users\\ianto\\Desktop\\Corso Python per l analisi dei dati\\File\\")

In [ ]:
x1

In [ ]:
w, lista_errori = perceptron(x = x1, y = y1)

In [ ]:
w

In [ ]:
predici(x = x2[0],w = w), y2[0]

# Perceptron e programmazione a oggetti - fine parte 1


# Perceptron e programmazione a oggetti - inizio parte 2

# Creiamo la classe perceptron

In [ ]:
class perceptron_class(object):
    def __init__(self,eta = 0.1,epoche = 10):
        self.eta = eta
        self.epoche = epoche
    
    def training(self,x,y):
        '''
        x è un array bidimensionale di numpy contenente più righe di features
        y è un array bidimensionale di numpy contenente più righe di classi
        '''
        self.w = np.zeros(x.shape[1])
        self.listaErrori = []
        iterazioniEseguite = 0
        for it in range(self.epoche):
            iterazioniEseguite = iterazioniEseguite + 1
            errori = 0
            for i in range(x.shape[0]):
                predizione = self.predict(x[i])
                self.w = self.w + self.eta * (y[i]-predizione) * x[i]
                if (y[i]-predizione) != 0:
                    errori = errori + 1
            self.listaErrori.append(errori)  
            if(errori == 0):
                break 
    
    def predict(self,x):
        '''
        x è un'array monodimensionale di numpy con una sola riga
        '''
        xw = np.dot(x,self.w) 
        if xw >= 0:
            return 1
        else:
            return -1
        
    def evaluate(self,x,y):
        '''
        x è un array bidimensionale di numpy contenente più righe di features
        y è un array bidimensionale di numpy contenente più righe di classi
        '''
        self.predizioniTest = []
        self.erroriTest = 0
        for i in range(x.shape[0]):
            predizione = self.predict(x[i])
            self.predizioniTest.append(predizione)
            if(predizione != y[i]):
                self.erroriTest = self.erroriTest + 1 
        self.accuratezza = 1 - self.erroriTest/len(x)

## Utilizziamo la classe appena creata

Istanziamo l'oggetto

In [ ]:
pcp = perceptron_class(epoche = 5)

Visualizziamo l'attributo epoche valorizzato nel metodo costruttore

In [ ]:
pcp.epoche

In [ ]:
pcp.eta

Il metodo costruttore non valorizza l'attributo w, di conseguenza la prossima istruzione genera un errore

In [ ]:
pcp.w

Chiamiamo il metodo training dando in input i vettori x1 e y1 creati in precedenza con il preprocessing

In [ ]:
pcp.training(x1,y1)

A questo punto l'attributo w sarà valorizzato

In [ ]:
pcp.w

lanciamo il metodo evaluate e visualizziamo l'attributo accuratezza

In [ ]:
pcp.evaluate(x2,y2)

In [ ]:
pcp.accuratezza

# Perceptron e programmazione a oggetti - fine parte 2

# Perceptron e programmazione a oggetti - inizio parte 3

# Creiamo la classe preprocessing_iris

In [ ]:
class preprocessing_iris(object):
    
    def __init__(self,path):
        self.path = path
     
    def work_on_iris(self,percentuale = 0.5, percentuale_training = 0.7):
        
        self.import_iris(path = self.path)
        
        #copia, filtro e mescolamento
        iris_working = self.iris.copy()
        iris_working = iris_working[iris_working["class"].isin(["Iris setosa","Iris versicolor"])]
        iris_working = iris_working.sample(frac=1,
                                           random_state = 0)
        iris_working = iris_working.drop(["rownumber"], 
                                     axis=1)
        #calcolo colonne da eliminare
        numero_righe = len(iris_working)
        soglia = numero_righe*percentuale
        lista_colonne = list(iris_working.columns)
        lista_colonne.remove("class")
        elenco_colonne_da_eliminare = []
        for colonna in lista_colonne:
            numero_null = len(iris_working[iris_working[colonna].isna()])
            if numero_null > soglia:
                lista_colonne.remove(colonna)
                elenco_colonne_da_eliminare.append(colonna)
        
        #rimozione colonne con null
        self.rimuovi_colonne(iris_working,elenco_colonne_da_eliminare)
        
        
        #codifica etichette
        iris_working.loc[iris_working["class"] == "Iris setosa", "class"] = 1
        iris_working.loc[iris_working["class"] == "Iris versicolor", "class"] = -1

        #training e test
        numero_righe_training = int(numero_righe*percentuale_training)
        iris_training = iris_working.iloc[0:numero_righe_training].copy()
        iris_test = iris_working.iloc[numero_righe_training:].copy()
        
        #calcolo medie e deviazione standard
        elenco_medie = []    
        elenco_deviazioni_standard = [] 
        for colonna in lista_colonne:
            media = iris_training[colonna].mean()
            elenco_medie.append(media)
            deviazione_standard = iris_training[colonna].std()
            elenco_deviazioni_standard.append(deviazione_standard)
            
        #valorizza null
        self.valorizza_null(iris_training,lista_colonne,elenco_medie)
        self.valorizza_null(iris_test,lista_colonne,elenco_medie)
        
        #normalizzazione
        self.normalizzazione(iris_training, lista_colonne, elenco_medie, elenco_deviazioni_standard)
        self.normalizzazione(iris_test, lista_colonne, elenco_medie, elenco_deviazioni_standard)
        
        #aggiunta colonna
        self.aggiunta_colonna(iris_training)
        self.aggiunta_colonna(iris_test)
        
        #pubblicazione attributi
        self.lista_colonne = lista_colonne
        self.elenco_medie = elenco_medie
        self.elenco_deviazioni_standard = elenco_deviazioni_standard
        self.elenco_colonne_da_eliminare = elenco_colonne_da_eliminare
        self.iris_training = iris_training.copy()
        self.iris_test = iris_test.copy()
        self.x1 = iris_training[["sepal_length","sepal_width","petal_length","petal_width","weight"]].values
        self.y1 = iris_training[["class"]].values
        self.x2 = iris_test[["sepal_length","sepal_width","petal_length","petal_width","weight"]].values
        self.y2 = iris_test[["class"]].values
        

    def import_iris(self,path):
        self.iris = pd.read_csv(filepath_or_buffer= self.path + 'IrisDataset.csv',
                sep = ";", 
                header = None,
                names = ["rownumber","sepal_length","sepal_width","petal_length","petal_width","class"],
                skiprows = 4
                )
 
    def rimuovi_colonne(self, x, colonne):
        '''
        x è un dataframe
        colonne è una lista di colonne
        '''
        for colonna in colonne:
            x = x.drop(colonna, axis=1)  

    def valorizza_null(self, x, colonne, medie):
        '''
        x è un dataframe
        colonne è una lista di stringhe (contenenti i nomi delle colonne)
        medie è una lista di float (contenenti le medie delle rispettive colonne)
        '''
        for colonna, media in zip(colonne, medie):
            x[colonna] = x[colonna].fillna(media)
            
    def normalizzazione(self, x, colonne, medie, deviazioni_standard):
        '''
        x è un dataframe
        colonne è una lista di stringhe (contenenti i nomi delle colonne)
        medie è una lista di float (contenenti le medie delle rispettive colonne)
        deviazioni_standard è una lista di float (contenenti le deviazioni standard delle rispettive colonne)
        '''
        for colonna, media, deviazione_standard in zip(colonne, medie, deviazioni_standard):
            x[colonna] = (x[colonna] - media) / deviazione_standard
    
    def aggiunta_colonna(self, x):
        '''
        x è un dataframe
        '''
        x["weight"] = 1
        
    def preprocessing_new_row(self,x):
        '''
        x è un array monodimensionale o bidimensionale di numpy contenente una sola riga di features
        '''    
        if type(x) is np.ndarray:
            if type(x[0]) is np.ndarray:
                df = pd.DataFrame(data = list(x), 
                          columns = self.lista_colonne)
            else: 
                df = pd.DataFrame(data = [list(x)], 
                          columns = self.lista_colonne) 
        df_working = df.copy()
        self.rimuovi_colonne(df_working, self.elenco_colonne_da_eliminare)
        self.valorizza_null(df_working, self.lista_colonne, self.elenco_medie)
        self.normalizzazione(df_working, self.lista_colonne, self.elenco_medie, self.elenco_deviazioni_standard)
        self.aggiunta_colonna(df_working)
        self.new_row = df_working.values               

Inizializziamo l'oggetto pre dalla classe preprocessing_iris

In [ ]:
pre = preprocessing_iris(path)

lanciamo il metodo work_on_iris

In [ ]:
pre.work_on_iris()

Il metodo valorizza gli attributi x1, x2, y1 e y2 con cui eseguirò l'algoritmo Perceptron

In [ ]:
len(pre.x1), len(pre.x2), len(pre.y1), len(pre.y2)

Istanzio l'oggetto pcp relativo alla classe perceptron_class

In [ ]:
pcp = perceptron_class()

Eseguo il metodo di training dando in input gli attributi x1 e y1 dell'oggetto pre

In [ ]:
pcp.training(pre.x1,pre.y1)

Visualizzo l'attributo w generato

In [ ]:
pcp.w

Rivaluto l'accuratezza dando in input gli attributi x2 e y2 dell'oggetto pre

In [ ]:
pcp.evaluate(pre.x2,pre.y2)

In [ ]:
pcp.accuratezza

A questo punto posso usare l'algoritmo su una nuova riga

In [ ]:
riga = np.array([3.9,3.4, 1.5, 0.3])

Eseguo le operazioni di preprocessing su questa riga utilizzando metodi e attributi della classe preprocessing_iris, valorizzati in precedenza nell'oggetto pre a partire dal file Iris

In [ ]:
pre.preprocessing_new_row(riga)

Visualizziamo come appare la nuova riga dopo il preprocessing interrogando l'attributo new_row

In [ ]:
pre.new_row

Utilizzo il metodo predict della classe perceptron_class per predirre la classe della nuova riga. Chiamando il metodo sull'oggetto pcp, per il calcolo della predizione verrà utilizzato l'attributo pcp.w generato nelle istruzioni precedenti.

In [ ]:
pcp.predict(pre.new_row)